In [93]:
from pathlib import Path
import pandas as pd
import numpy as np
import data

path_to_raw_data = Path("../src/data/raw")
path_to_processed_data = Path("../src/data/processed")
path_to_copilot_table = path_to_raw_data / "delta_change_copilot.csv"

In [30]:
# Read the delta change method values from the GEUS table
df_copilot = pd.read_csv(path_to_copilot_table)
df_copilot = df_copilot[np.logical_and.reduce((df_copilot["region"]=="DK5", df_copilot["scenario"]=="far", df_copilot["metric"]=="dc"))]
df_copilot.drop(["region", "scenario", "metric", "page"], axis=1, inplace=True)
df_copilot["month"] = np.select(
    [
        df_copilot["month"]=="Jan",
        df_copilot["month"]=="Feb",
        df_copilot["month"]=="Mar",
        df_copilot["month"]=="Apr",
        df_copilot["month"]=="May",
        df_copilot["month"]=="Jun",
        df_copilot["month"]=="Jul",
        df_copilot["month"]=="Aug",
        df_copilot["month"]=="Sep",
        df_copilot["month"]=="Oct",
        df_copilot["month"]=="Nov",
        df_copilot["month"]=="Dec",
    ],
    np.arange(1,13)
)

In [96]:
# Apply delta changes (factor for precipitation/epot and absolute for temperature)
precipitation_future = data.timeseries[["date", "month", "precipitation"]].copy()
precipitation_future = precipitation_future.iloc[:-1] # Drop the last day, because it needs to match future period length
precipitation_future["date_future"] = pd.date_range("2071-01-01", "2100-12-31")
precipitation_future = precipitation_future.merge(
    df_copilot[df_copilot["variable"]=="Precipitation"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
precipitation_future["precipitation_future"] = precipitation_future["precipitation"] * precipitation_future["value"]

evapotranspiration_future = data.timeseries[["date", "month", "epot"]].copy()
evapotranspiration_future = evapotranspiration_future.iloc[:-1] # Drop the last day, because it needs to match future period length
evapotranspiration_future["date_future"] = pd.date_range("2071-01-01", "2100-12-31")
evapotranspiration_future = evapotranspiration_future.merge(
    df_copilot[df_copilot["variable"]=="Reference ET"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
evapotranspiration_future["epot_future"] = evapotranspiration_future["epot"] * evapotranspiration_future["value"]

temperature_future = data.timeseries[["date", "month", "temperature"]].copy()
temperature_future = temperature_future.iloc[:-1] # Drop the last day, because it needs to match future period length
temperature_future["date_future"] = pd.date_range("2071-01-01", "2100-12-31")
temperature_future = temperature_future.merge(
    df_copilot[df_copilot["variable"]=="Temperature"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
temperature_future["temperature_future"] = temperature_future["temperature"] + temperature_future["value"]

# Merge results, save to csv
future = precipitation_future[["date_future", "model", "precipitation_future"]].merge(
    temperature_future[["date_future", "model", "temperature_future"]].merge(
        evapotranspiration_future[["date_future", "model", "epot_future"]],
        how="inner",
        on=["date_future", "model"]
    ),
    how="inner",
    on=["date_future", "model"]
).rename({
    "date_future": "date",
    "precipitation_future": "precipitation",
    "temperature_future": "temperature",
    "epot_future": "epot",
}, axis=1)
future.to_csv(path_to_processed_data / "timeseries_future_deltachange.csv", index=False)

In [82]:
def ingest_model(df, model):
    df = df.copy()
    df["model"] = model
    return df

precip_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_raw_data / "Precip_DBS.xlsx", sheet_name=None).items()])
et_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_raw_data / "RefET_DBS.xlsx", sheet_name=None).items()])
temp_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_raw_data / "Temp_DBS.xlsx", sheet_name=None).items()])

dbs = precip_dbs.merge(
    et_dbs.merge(
        temp_dbs,
        on=["Time", "model"],
        how="inner"
    ),
    on=["Time", "model"],
    how="inner"
)
dbs.rename({
    "Time": "date",
    "Precip (mm)": "precipitation",
    "RefET (mm)": "epot",
    "Temp (°C)": "temperature",
}, axis=1, inplace=True)
dbs.to_csv(path_to_processed_data / "timeseries_future_dbs.csv", index=False)